# Import Library

In [2]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('operations').getOrCreate()

from pyspark.sql.functions import col, sum as sum_

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/02 10:04:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1 - BU

The detial information about BU is addressed in Itertion-4 report Section - 1.

# 2- DU

In [3]:
# load the csv file
df = spark.read.csv('Datasets/heart_dataset_1.csv', inferSchema=True, header=True)

df.count()

69000

In [4]:
# Verify the loaded data
df.show()

+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|index| id|  age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|    0|  0|18393|     2|   168|  62.0|  110|   80|          1|   1|    0|   0|     1|     0|
|    1|  1|20228|     1|   156|  85.0|  140|   90|          3|   1|    0|   0|     1|     1|
|    2|  2|18857|     1|   165|  64.0|  130|   70|          3|   1|    0|   0|     0|     1|
|    3|  3|17623|     2|   169|  82.0|  150|  100|          1|   1|    0|   0|     1|     1|
|    4|  4|48Yrs|     1|   156|  56.0|  100|   60|          1|   1|    0|   0|     0|     0|
|    5|  8|21914|     1|   151|  67.0|  120|   80|          2|   2|    0|   0|     0|     0|
|    6|  9|22113|     1|   157|  93.0|  130|   80|          3|   1|    0|   0|     1|     0|
|    7| 12|22584|     2|   178|  95.0|  130|   90|          3|   3|   

In [5]:
# Verify the data schema
df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- ap_hi: integer (nullable = true)
 |-- ap_lo: integer (nullable = true)
 |-- cholesterol: integer (nullable = true)
 |-- gluc: integer (nullable = true)
 |-- smoke: integer (nullable = true)
 |-- alco: integer (nullable = true)
 |-- active: integer (nullable = true)
 |-- cardio: integer (nullable = true)



In [21]:
df.columns

['index',
 'id',
 'age',
 'gender',
 'height',
 'weight',
 'ap_hi',
 'ap_lo',
 'cholesterol',
 'gluc',
 'smoke',
 'alco',
 'active',
 'cardio']

In [22]:
df.head(10)

[Row(index=0, id=0, age='18393', gender=2, height=168, weight=62.0, ap_hi=110, ap_lo=80, cholesterol=1, gluc=1, smoke=0, alco=0, active=1, cardio=0),
 Row(index=1, id=1, age='20228', gender=1, height=156, weight=85.0, ap_hi=140, ap_lo=90, cholesterol=3, gluc=1, smoke=0, alco=0, active=1, cardio=1),
 Row(index=2, id=2, age='18857', gender=1, height=165, weight=64.0, ap_hi=130, ap_lo=70, cholesterol=3, gluc=1, smoke=0, alco=0, active=0, cardio=1),
 Row(index=3, id=3, age='17623', gender=2, height=169, weight=82.0, ap_hi=150, ap_lo=100, cholesterol=1, gluc=1, smoke=0, alco=0, active=1, cardio=1),
 Row(index=4, id=4, age='48Yrs', gender=1, height=156, weight=56.0, ap_hi=100, ap_lo=60, cholesterol=1, gluc=1, smoke=0, alco=0, active=0, cardio=0),
 Row(index=5, id=8, age='21914', gender=1, height=151, weight=67.0, ap_hi=120, ap_lo=80, cholesterol=2, gluc=2, smoke=0, alco=0, active=0, cardio=0),
 Row(index=6, id=9, age='22113', gender=1, height=157, weight=93.0, ap_hi=130, ap_lo=80, cholestero

In [26]:
df.orderBy(df['Index'].asc()).show(10)

+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|index| id|  age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|    0|  0|18393|     2|   168|  62.0|  110|   80|          1|   1|    0|   0|     1|     0|
|    1|  1|20228|     1|   156|  85.0|  140|   90|          3|   1|    0|   0|     1|     1|
|    2|  2|18857|     1|   165|  64.0|  130|   70|          3|   1|    0|   0|     0|     1|
|    3|  3|17623|     2|   169|  82.0|  150|  100|          1|   1|    0|   0|     1|     1|
|    4|  4|48Yrs|     1|   156|  56.0|  100|   60|          1|   1|    0|   0|     0|     0|
|    5|  8|21914|     1|   151|  67.0|  120|   80|          2|   2|    0|   0|     0|     0|
|    6|  9|22113|     1|   157|  93.0|  130|   80|          3|   1|    0|   0|     1|     0|
|    7| 12|22584|     2|   178|  95.0|  130|   90|          3|   3|   

In [23]:
df.tail(10)

[Row(index=68990, id=98559, age='20508', gender=2, height=173, weight=91.0, ap_hi=140, ap_lo=90, cholesterol=1, gluc=1, smoke=0, alco=1, active=1, cardio=0),
 Row(index=68991, id=98561, age='21787', gender=1, height=168, weight=70.0, ap_hi=140, ap_lo=90, cholesterol=3, gluc=1, smoke=0, alco=0, active=1, cardio=1),
 Row(index=68992, id=98562, age='17574', gender=1, height=157, weight=63.0, ap_hi=150, ap_lo=100, cholesterol=1, gluc=1, smoke=0, alco=0, active=1, cardio=0),
 Row(index=68993, id=98563, age='14643', gender=2, height=175, weight=75.0, ap_hi=100, ap_lo=70, cholesterol=1, gluc=1, smoke=0, alco=0, active=1, cardio=0),
 Row(index=68994, id=98564, age='18949', gender=2, height=172, weight=62.0, ap_hi=110, ap_lo=70, cholesterol=1, gluc=1, smoke=1, alco=0, active=1, cardio=0),
 Row(index=68995, id=98565, age='17548', gender=2, height=173, weight=72.0, ap_hi=120, ap_lo=80, cholesterol=1, gluc=1, smoke=0, alco=0, active=0, cardio=0),
 Row(index=68996, id=98566, age='20430', gender=1, 

In [25]:
df.orderBy(df['Index'].desc()).show(10)

+-----+-----+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|index|   id|  age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+-----+-----+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|68999|98569|19591|     2|   175|  88.0|  120|   80|          1|   1|    0|   0|     0|     1|
|68998|98568|19107|     1|   154|  77.0|   14|   90|          1|   1|    0|   0|     1|     0|
|68997|98567|21682|     1|   158|  78.0|  130|   90|          1|   1|    0|   0|     1|     1|
|68996|98566|20430|     1|   159| 105.0|  120|   80|          1|   1|    0|   0|     1|     1|
|68995|98565|17548|     2|   173|  72.0|  120|   80|          1|   1|    0|   0|     0|     0|
|68994|98564|18949|     2|   172|  62.0|  110|   70|          1|   1|    1|   0|     1|     0|
|68993|98563|14643|     2|   175|  75.0|  100|   70|          1|   1|    0|   0|     1|     0|
|68992|98562|17574|     1|   157|  63.0|  150|  10

In [24]:
df.select('index','id').describe().show()

df.select('age','gender','height','weight').describe().show()

df.select('ap_hi','ap_lo','cholesterol','gluc').describe().show()

df.select('smoke','alco','active','cardio').describe().show()

+-------+------------------+------------------+
|summary|             index|                id|
+-------+------------------+------------------+
|  count|             69000|             69000|
|   mean|           34499.5| 49257.99002898551|
| stddev|19918.728624086427|28438.175895091368|
|    min|                 0|                 0|
|    max|             68999|             98569|
+-------+------------------+------------------+

+-------+------------------+-------------------+------------------+-----------------+
|summary|               age|             gender|            height|           weight|
+-------+------------------+-------------------+------------------+-----------------+
|  count|             69000|              69000|             69000|            69000|
|   mean|19469.362175167036| 1.3495217391304348|164.35059420289855|74.21126086956522|
| stddev|2467.2731299236907|0.47682238629511076| 8.211304686499416|14.40431106828465|
|    min|             10798|                  1|   

In [7]:
# Check for missing values in each column
missing_values = df.agg(*[(sum_(col(c).isNull().cast("int"))).alias(c) for c in df.columns])

missing_values.show()

+-----+---+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|index| id|age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+-----+---+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|    0|  0|  0|     0|     0|     0|    0|    0|          3|   0|    0|   0|     0|     0|
+-----+---+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+



# 3 - DP

# 4 - DT

# 5 - DMM

The detial information about DMM is addressed in Itertion-4 report Secion - 5

# 6 - DMA

# 7 - DM

# 8 - INT

# 9 - ACT

Desribe the Action Plan to Implement, Observe and Improve